In [2]:
import sys
sys.path.append('../../src')
from modules import *
from columns import *
sys.path.append('../../src/python')
from random_funks import *

# Dataframe sa 3 modela za LC50

In [3]:
# DAPHNIA MAGNA EPA # mg/L
report_DM=pd.read_csv('../../data/VEGA_QSAR_modeli/report_DAPHNIA_EPA.txt',index_col=0, sep='\t')
report_DM = zamijeni_nazive_molekula(report_DM)
Daphnia_Magna_EPA = report_DM['Assessment'].values

In [4]:
# FATHEAD MINNOW EPA # mg/L
report_FM=pd.read_csv('../../data/VEGA_QSAR_modeli/report_FATHEAD_EPA.txt',index_col=0, sep='\t')
Fathead_Minnow_EPA = report_FM['Assessment'].values

In [5]:
# FISH ACUTE # mg/L
report_FA=pd.read_csv('../../data/VEGA_QSAR_modeli/report_FISH_LC50.txt',index_col=0, sep='\t')
Fish_acute_LC50 = report_FA['Assessment'].values

In [6]:
LC50_toks = pd.DataFrame({'Id': report_DM['Id'], 
                     'Daphnia_Magna_EPA_mg/L': Daphnia_Magna_EPA, 
                     'Fathead_Minnow_EPA_mg/L': Fathead_Minnow_EPA, 
                     'Fish_acute_LC50_mg/L': Fish_acute_LC50})
LC50_toks_modified = extract_numeric_from_df(LC50_toks)

In [7]:
LC50_toks_modified.to_csv('../../data/VEGA_QSAR_modeli/LC50_toks.csv', index=False)

# Izračun TU_site za sva 3 modela pojedinačno

In [8]:
data=pd.read_csv('../../data/PAH_PCB_OCP.csv',index_col=0)
report=pd.read_csv('../../data/VEGA_QSAR_modeli/LC50_toks.csv',index_col=0)
report_logP=pd.read_csv('../../data/VEGA_QSAR_modeli/logP_median.csv')

In [9]:
# DAPHNIA MAGNA EPA # mg/L
TU_site_DM = racunaj_TU_site_log10(data, report, report_logP, 0)

In [10]:
# FATHEAD MINNOW EPA # mg/L
TU_site_FM = racunaj_TU_site_log10(data, report, report_logP, 1)

In [11]:
# FISH ACUTE # mg/L
TU_site_FA = racunaj_TU_site_log10(data, report, report_logP, 2)

In [12]:
TU_site_LC50 = pd.DataFrame({'LOKACIJA': data['LOKACIJA'],
                             'RIJEKA': data['RIJEKA'],
                     'TU_site_DM': TU_site_DM, 
                     'TU_site_FM': TU_site_FM, 
                     'TU_site_FA': TU_site_FA})

# Rangirala svaki posebno i average rank

In [13]:
# Rangiranje TU_site vrijednosti za svaki model
TU_site_LC50['TU_site_DM_rank'] = TU_site_LC50['TU_site_DM'].rank(ascending=False)
TU_site_LC50['TU_site_FM_rank'] = TU_site_LC50['TU_site_FM'].rank(ascending=False)
TU_site_LC50['TU_site_FA_rank'] = TU_site_LC50['TU_site_FA'].rank(ascending=False)

# Izračunavanje prosječnog ranga
average_rank = (TU_site_LC50['TU_site_DM_rank'] + TU_site_LC50['TU_site_FM_rank'] + TU_site_LC50['TU_site_FA_rank']) / 3

TU_site_LC50['TU_site_average_rank'] = average_rank
data['TU_site_average_rank'] = average_rank

In [14]:
TU_site_LC50.to_csv('../../data/VEGA_QSAR_modeli/LC50_toks_average.csv', index=False)

# PLOT NA KARTU

In [15]:
# PROBA

mapa = folium.Map(location=[data['LAT'].mean(), data['LON'].mean()], zoom_start=7)

# Dodavanje markera za svaku lokaciju
for index, row in data.iterrows():
    # Određivanje boje na temelju ranga
    if row['TU_site_average_rank'] <= 5:
        color = 'red'
    elif row['TU_site_average_rank'] <= 10:
        color = 'orange'
    else:
        color = 'green'

    folium.CircleMarker(
        location=[row['LAT'], row['LON']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        popup=f"Lokacija: {index}<br>Prosj. rang toksičnosti: {row['TU_site_average_rank']:.2f}"
    ).add_to(mapa)

# Prikazivanje mape
#mapa.save('mapa_toksicnosti.html')

mapa

# TU_sed sa FATHEAD MINNOW

In [16]:
TU_sed_FM = racunaj_TU_sed(data, report, report_logP, 1)
TU_sed_FM = pd.DataFrame(TU_sed_FM, columns=PAH_kol+OCP_kol+PCB_kol)

TU_sed_FM['LOKACIJA'] = data['LOKACIJA']
TU_sed_FM['RIJEKA'] = data['RIJEKA']

In [17]:
TU_sed_FM.to_csv('../../data/VEGA_QSAR_modeli/TU_sed_FM.csv', index=False)

In [37]:
TU_sed_FM = racunaj_TU_sed(data, report, report_logP, 1)
TU_sed_FM = pd.DataFrame(TU_sed_FM, columns=PAH_kol+OCP_kol+PCB_kol)

In [52]:
TU_sed_FM_rank = TU_sed_FM.T.rank(ascending=False)
TU_sed_FM_mean = TU_sed_FM_rank.mean(axis=1) # mean za svaku molekulu

# TU_sed sa DAPHNIA MAGNA

In [53]:
TU_sed_DM = racunaj_TU_sed(data, report, report_logP, 0)
TU_sed_DM = pd.DataFrame(TU_sed_DM, columns=PAH_kol+OCP_kol+PCB_kol)

In [54]:
TU_sed_DM_rank = TU_sed_DM.T.rank(ascending=False)
TU_sed_DM_mean = TU_sed_DM_rank.mean(axis=1) 

# TU_sed sa FISH ACUTE

In [55]:
TU_sed_FA = racunaj_TU_sed(data, report, report_logP, 2)
TU_sed_FA = pd.DataFrame(TU_sed_FA, columns=PAH_kol+OCP_kol+PCB_kol)

In [56]:
TU_sed_FA_rank = TU_sed_FA.T.rank(ascending=False)
TU_sed_FA_mean = TU_sed_FA_rank.mean(axis=1)

# T za PBT_score

In [77]:
toks_rank = pd.DataFrame({
                     'TU_sed_DM_mean': TU_sed_DM_mean, 
                     'TU_sed_FM_mean': TU_sed_FM_mean, 
                     'TU_sed_FA_mean': TU_sed_FA_mean})

In [78]:
toks_rank.to_csv('../../data/VEGA_QSAR_modeli/toks_rank.csv', index=False)